In [1]:
from radiomics.featureextractor import RadiomicsFeatureExtractor 
import SimpleITK as sitk 
import os 
import numpy as np 
from library_dicom.dicom_processor.tools.pyradiomic import *

In [2]:
pet_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/51011102171008/pet0/1.2.276.0.7230010.3.2.301136_nifti_PT.nii'
mask_path =  '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/51011102171008/pet0/1.2.276.0.7230010.3.2.301136_nifti_mask.nii'

pet_img = sitk.ReadImage(pet_path)

mask_img = sitk.ReadImage(mask_path)
mask_array = sitk.GetArrayFromImage(mask_img)
print(mask_array.shape)

(5, 287, 168, 168)


In [2]:
folder = '/media/deeplearning/CLé USB/CenterOfMAss'
liste_nifti = os.listdir(folder)
liste_nifti.remove('salim10.xlsx')

dataset = []
for nifti in liste_nifti :
    dataset.append(os.path.join(folder, nifti))
    

In [3]:
dataset

['/media/deeplearning/CLé USB/CenterOfMAss/11009101026007_feb 14_2011ASC_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026020_apr 2_2013.ASCcsv_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026025_oct 7_2013.csvLV_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101036003_jun 12_2009.csvLV_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101036013_apr 6_2010.csvLV1_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026017_jan 14_2013ASC_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026013_apr 19_2012.csvASC_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026012_mar 5_2012ASC_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026002_jun 16_2009ASC_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026021_may 3_2013.csvLV_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/.~lock.salim10.xlsx#']

In [5]:
distance_max = []
for nifti in dataset : 
    print(nifti)
    liste_center = get_center_of_mass(nifti)
    d_max = calcul_distance_max(liste_center)
    distance_max.append(d_max)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

RuntimeError: Exception thrown in SimpleITK ReadImage: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:107:
sitk::ERROR: Unable to determine ImageIO reader for "/media/deeplearning/CLé USB/CenterOfMAss/.~lock.salim10.xlsx#"

In [8]:
dataset

['/media/deeplearning/CLé USB/CenterOfMAss/11009101026007_feb 14_2011ASC_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026020_apr 2_2013.ASCcsv_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026025_oct 7_2013.csvLV_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101036003_jun 12_2009.csvLV_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101036013_apr 6_2010.csvLV1_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026017_jan 14_2013ASC_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026013_apr 19_2012.csvASC_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026012_mar 5_2012ASC_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026002_jun 16_2009ASC_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/11009101026021_may 3_2013.csvLV_pmask.nii',
 '/media/deeplearning/CLé USB/CenterOfMAss/.~lock.salim10.xlsx#']

In [9]:
distance_max

[687.9410017961078,
 409.35030941124745,
 601.106305139214,
 690.2595821157842,
 399.7400060778472,
 714.7205005645186,
 628.9593775075031,
 141.06434745309886,
 723.6325464375252,
 784.6480262610567]